## Exploration with Data and Visuals
#### Gavin Qu 11-09-2025

### 1. API Config

In [ ]:
# config.py, for now just this cell
USERNAME = "gavinqu1@gmail.com-api-client"
PASSWORD = "NbB46EFxJZO2OevTVHAlgK7r1BxhOqZh"

from opensky_api import OpenSkyApi
api = OpenSkyApi(USERNAME, PASSWORD)

# Example: Around London, UK
# (Min Lon, Max Lon, Min Lat, Max Lat)
# BBOX_LONDON = (-2.0, 3.0, 50.0, 52.0)

### 2. Historical Data

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

url = "https://zenodo.org/record/8360680/files/2023-10-01_states.csv.gz"
df = pd.read_csv(url, compression='gzip', nrows=10000)  # sample subset for speed

df = df.rename(columns={"time": "timestamp", "baroaltitude": "baro_altitude"})
gdf = gpd.GeoDataFrame(
    df.dropna(subset=["longitude", "latitude"]),
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
)

gdf.head()